# Section 1 : VectorDB Operations with Faiss

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain import OpenAI, VectorDBQA
from langchain.chains import RetrievalQAWithSourcesChain
import pandas as pd
import os
from dotenv import load_dotenv # openai api key 입력을 위해서 필요

In [2]:
OPENAI_API_KEY_PATH = "/Users/eugene/Dropbox/0_Dev/07_LLM/project/env_folder/tonchat_key/.env"
# 셋째, .env 파일을 현재 실행환경에 등록. 그 결과, 개별 API 함수의 인자에 KEY값을 일일이 넣지 않아도 됨
load_dotenv(OPENAI_API_KEY_PATH)
# 넷째, 현재 실행환경에서 "OPENAI_API_KEY"라고 명명된 값을 불러오기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
###########################################
#       pdf 원본을 담은 loader 객체 생성       #
###########################################

# 전처리전의 data 폴더
data = "./data/"
# PyPDFLoader as the class method to load the files
loader = DirectoryLoader(data, loader_cls=PyPDFLoader)

###########################################
#          character 단위로 분할             #
###########################################

# load_and_split() 메소드를 호출하여 문서를 로드하고 page 단위로 분할저장
pages = loader.load_and_split()

# CharacterTextSplitter 객체 생성
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# CharacterTextSplitter 객체에 pages를 인자로 넣어서, character로 쪼개서 저장
docs = text_splitter.split_documents(pages)

embeddings = OpenAIEmbeddings()

/Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [4]:
# 현재 파이썬 라이브러리가 설치된 장소를 알아보자
import sys
sys.path

['/Users/eugene/Dropbox/0_Dev/07_LLM/project/tonchat',
 '/Users/eugene/opt/anaconda3/envs/LLM/lib/python310.zip',
 '/Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10',
 '/Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/lib-dynload',
 '',
 '/Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/site-packages']

In [5]:
# 위의 경로에 패키지들이 제대로 설칭되었는지 확인
!pip list -v

Package                                  Version    Location                                                          Installer
---------------------------------------- ---------- ----------------------------------------------------------------- ---------
aiofiles                                 23.2.1     /Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/site-packages pip
aiohttp                                  3.9.3      /Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/site-packages pip
aiosignal                                1.3.1      /Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/site-packages pip
altair                                   5.2.0      /Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/site-packages pip
annotated-types                          0.6.0      /Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/site-packages pip
anyio                                    4.2.0      /Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/site-packages pip
appnope     

In [4]:
###########################################
#              vector DB 생성              #
###########################################

# docs를 embedding model을 이용하여 vector DB로 변환
db = FAISS.from_documents(docs, embeddings)

In [60]:
print(dir(db))

['_FAISS__add', '_FAISS__from', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_aembed_documents', '_aembed_query', '_asimilarity_search_with_relevance_scores', '_cosine_relevance_score_fn', '_embed_documents', '_embed_query', '_euclidean_relevance_score_fn', '_get_retriever_tags', '_max_inner_product_relevance_score_fn', '_normalize_L2', '_select_relevance_score_fn', '_similarity_search_with_relevance_scores', 'aadd_documents', 'aadd_texts', 'add_documents', 'add_embeddings', 'add_texts', 'adelete', 'afrom_documents', 'afrom_embeddings', 'afrom_texts', 'amax_marginal_relevance_search', 'amax_marginal_relevance_search_by_vector', 'amax_marginal_

## 01. similarity search

In [7]:
# similarity_search(query) 메소드를 이용 : SQL언어가 아닌 자연어인 query와 가장 유사한 문서를 찾아냄
query = 'fee'
docs_result = db.similarity_search(query)
docs_result

[Document(page_content="24. 1. 4. 오전  10:49 L2 fee - (current) User Guide\nhttps://tokamaknetwork.gitbook.io/home/02-service-guide/titan/user-guide/l2-fee 1/1L2 fee\nThis document outlines the calculation process for transaction fees on Titan, emphasizing their lower\ncost compared to Ethereum.\nTitan's transaction fees are determined by adding two components together:\nL2 execution fee\nL1 security fee\nL2 execution fee\ntx.gasPrice * l2GasUsed\nThe L2 execution fee refers to the fee incurred when a user's transaction is processed within the\nTitan layer2 network. It follows a calculation method shown above, identical to the standard\nEthereum transaction fee computation. However, the distinction lies in the fact that the gas price on\nTitan is significantly lower compared to Ethereum.\nL1 security fee\nl1_base_fee * (tx_data_gas + overhaed) * scalar\nThe L1 security fee is an essential fee required in the context of Optimistic Rollup. With Optimistic\nRollup, all L2 transactions are 

In [14]:
###########################################
#      db검색을 위한 retriever 객체를 생성      #
###########################################
retriever = db.as_retriever()


#############################################################################
#                  chain 모델 생성 : (llm, retriver)를  연결                    #
#############################################################################
# LLM : 사용자의 자연어를 retriever로 전달하고, retriever의 결과를 사용자에게 자연어로 반환
# OpenAI(model="gpt-3.5-turbo-instruct") 처럼 ()안에 사용할 모델을 명시하지 않으면 text-davinci-003을 사용하는데, 이는 legacy model이라 헌재 지원안됨
# 따라서 OpenAI(model="gpt-3.5-turbo-instruct") 처럼 ()안에 사용할 모델을 명시해야 함
# 지시사항을 수행하는 용도인 InstructGPT을 선택해야 하며, 대화용도의 ChatGPT 모델을 선택하면 안됨
model = RetrievalQAWithSourcesChain.from_chain_type(llm=OpenAI(model="gpt-3.5-turbo-instruct"), chain_type="stuff", retriever=retriever)

In [15]:
question = "metamask에 titan 네트워크를 추가하는 방법을 순서와 함께 상세히 알려주세요. "

# model() 메소드는 langchain 라이브러리의 RetrievalQAWithSourcesChain 클래스의 인스턴스 메서드
# 질의응답, context 등은 dictionary 자료형임
response = model({"question": question}, return_only_outputs=True)
print("Answer: ", response['answer'])
print("Source", response['sources'])

InvalidRequestError: This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?

## 02. Saving & loading 

In [8]:
#################################################
#     생성된 db를 로컬에 영구저장 : save_local()      #
#################################################

# vector bd 저장폴더
vector_directory = "test_vectordb"
# FAISS.save_local 메소드는 "test_vectordb" 폴더가 없으면 error 출력대신 해당폴더를 생성한다는 장점이 있다
# 해당폴더안에 기존의 index.faiss, index.pkl 파일이 있는 경우에는 덮어쓴다
db.save_local(vector_directory)

In [9]:
#######################################################
#       로컬에 저장된 DB를 불러오기 FAISS.load_local()      #
#######################################################

# (step 1: loading) 로컬 DB를 메모리에 로딩
db_loaded = FAISS.load_local(vector_directory, embeddings) #s aving, loading 둘다 폴더명을 인자로 받는다는 점을 주의해야 함 (파일명 x)

# (step 2: retriever 업데이트)
retriever_local = db.as_retriever()

# (step 3: chain 모델 업데이트)
model_local = RetrievalQAWithSourcesChain.from_chain_type(llm=OpenAI(model="gpt-3.5-turbo-instruct"), chain_type="stuff", retriever=retriever_local)

In [26]:
type(db_loaded)

langchain_community.vectorstores.faiss.FAISS

In [28]:
# db가 제대로 저장되었는지, 변경되었는지 파악하기 위해서 size를 측정해보는 것이 좋다
import sys
size_1 = sys.getsizeof(db_loaded)
size_2 = sys.getsizeof(db)
print("Size of db_loaded:", size_1, "bytes")
print("Size of db:", size_2, "bytes")


Size of db_local: 48 bytes
Size of db: 48 bytes


In [29]:
# dir()은 객체가 가지고 있는 속성과 메소드를 확인하는데 사용
# dir()조차도 객체가 가지고 있는 메소드 중의 하나
print(dir(db_loaded))

['_FAISS__add', '_FAISS__from', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_aembed_documents', '_aembed_query', '_asimilarity_search_with_relevance_scores', '_cosine_relevance_score_fn', '_embed_documents', '_embed_query', '_euclidean_relevance_score_fn', '_get_retriever_tags', '_max_inner_product_relevance_score_fn', '_normalize_L2', '_select_relevance_score_fn', '_similarity_search_with_relevance_scores', 'aadd_documents', 'aadd_texts', 'add_documents', 'add_embeddings', 'add_texts', 'adelete', 'afrom_documents', 'afrom_embeddings', 'afrom_texts', 'amax_marginal_relevance_search', 'amax_marginal_relevance_search_by_vector', 'amax_marginal_

## 03. View documenents

FAISS.from_documents(docs, embeddings) 생성자를 통해 생성된 FAISS 객체의 자료구조를 확인 --> vectorstore update에 필요

In [10]:
#####################################
#      vector db를 딕셔너리로 추출       #
#####################################

# vector db에 저장된 문서 전체를 추출해주는 docstore 객체
# docstore._dict 메소드를 사용하면 dictionary 자료형으로 추출
print(type(db.docstore._dict))

# 덕셔너리로 추출된 db --> 개별문서에서 chunking된 덩어리 (각 덩어리가 key, value 쌍으로 들어있음)
# key = 해당 chunk의 id
# value = 해당 chunk의 원래 문서의 제목, 문서에서의 페이지 등이 여러 개의 하위 dictionary로 저장됨
print(db.docstore._dict.keys())

<class 'dict'>
dict_keys(['17d1f794-9b23-401a-9705-ad0faeb9d637', 'ffe203f2-860e-41c2-8882-8376b8e98baf', 'd4a48108-2bc4-4e15-8bfb-137385307908', 'c321b2ad-1bf3-412d-9ff0-9409f788ce5b', 'e447f22f-9e87-4059-b361-c5e2f9754cc7', '031d0d69-78c4-45f4-916a-a6b2f11c6b7d', '4f6b2f1f-90b9-48ab-b482-704d2b6723d9', 'a83539f3-7834-4ddb-8561-d9338bdbf3bb', 'b99a06cc-1983-4d38-8a47-4b155bde37a8', '497d14b9-9745-4853-8e0b-ac54dba91c82', '6818a294-8fd1-4b04-b2d8-c810212a9bd3', 'cad1966d-7f13-416f-9a5d-dfda0294ccac', '424228b8-527b-4e01-9caa-27ed7057a748', '90fa7ffa-1cbd-49e5-aa07-29736b344448', '5eaa059f-73fe-405f-b7e8-57d3b1052ecc', 'ef2f3a12-d2ae-4b60-91be-45381f307b19', 'bc547c10-d0ea-4158-8bb6-7873a2692762', 'b79dc4cd-3d5d-4fe9-8ee2-56c4b6558228', '42697e1e-19dc-4d87-b365-062854f4ce40', '5b96b123-45e2-4552-a16a-082f73a9dc30', '084da1b3-a01e-4df7-8ab1-3131bf99c74c', 'e1c1cc04-da2b-4a9c-8511-8ab30d39018c', '9a6407d0-ede2-45f1-9bbc-49c1b199ad22', '403f9cbb-5dc8-4638-9db4-9e548d753fe2', 'aabc04c4-e199

In [11]:
#################################################
#         vector db를 dataframe으로 변환           #
#################################################

# vectorstore를 dataframe으로 변환 및 DB내 문서리스트 화면출력
def show_vstore(_store):
    vector_df = store_to_df(_store)
    display(vector_df)

# vectorstore를 dataframe으로 변환
def store_to_df(_store):
    #  docstore속성의 하위 속성인 _dict 속성을 이용하여 dictionary형으로 db전체를 추출
    v_dict = _store.docstore._dict
    print(type(v_dict)) # v_dict는 dictionary형
    data_rows = []

    # 추출된 v_db 딕셔너리는 많은 자료로 구성 --> 필요한 것만 key, value 쌍으로 추출하여 단순한 dictionary로 변환
    # v_dict의 key값은 db에 저장된 개별문서의 id값
    for k in v_dict.keys():
        # metadata는 vectorstore의 일부로서, 그 안에는 사전(dictionary) 형태의 데이터 구조를 가짐
        doc_name = v_dict[k].metadata['source'].split('/')[-1]
        page_number = v_dict[k].metadata['page']+1
        content = v_dict[k].page_content
        # k값에 대한 key를 chunk_id로 작명
        # 기타 3개의 값에도 key를 부여하여 dictionary로 완성 --> 데이터프레임 자료형인 data_rows에 추가
        data_rows.append({'chunk_id': k, 'document': doc_name, 'page': page_number, 'content': content})
    vector_df = pd.DataFrame(data_rows)
    return vector_df # 데이터프레임으로 변환된 vector db

# '''
# metadata는 faiss의 데이터 구조인 vectorstore의 일부로서, 그 안에는 사전(dictionary) 형태로 각 문서가 key, value 쌍으로 들어가 있음
# 사전의 key, value구조를 통해 문서의 출처(source), 페이지 번호(page)와 같은 문서에 대한 중요한 정보를 포함
# 이 정보를 통해 각 데이터 항목이 어디에서 왔는지, 어떤 특성을 가지고 있는지 등을 파악할 수 있습니다.
# '''

---
# 임시
#### index.faiss 파일을 만들어서 tonchat db 폴더에 직접 저장

In [13]:
###########################################
#       pdf 원본을 담은 loader 객체 생성       #
###########################################

# 전처리전의 data 폴더
data = "./data/"
# PyPDFLoader as the class method to load the files
loader = DirectoryLoader(data, loader_cls=PyPDFLoader)

###########################################
#               pdf --> text              #
###########################################

# load_and_split() 메소드를 호출하여 문서를 로드하고 page 단위로 분할저장
pages = loader.load_and_split()

# CharacterTextSplitter 객체 생성
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# CharacterTextSplitter 객체에 pages를 인자로 넣어서, character로 쪼개서 저장
docs = text_splitter.split_documents(pages)


###########################################
#          text --> vectorstore           #
###########################################

embeddings = OpenAIEmbeddings()

tonchat_db = FAISS.from_documents(docs, embeddings)

toncaht_db_folder = "vectorstore"
tonchat_db.save_local(toncaht_db_folder)

---

In [39]:
show_vstore(db)

<class 'dict'>


,chunk_id,document,page,content
0,56128eb0-5efb-4997-9d88-380fd9e2e26d,L2 fee.pdf,1,24. 1. 4. 오전 10:49 L2 fee - (current) User Gu...
1,d42f6dc2-ae7b-4a9b-b1d8-c29906ce4854,What is different_KR.pdf,1,24. 1. 4. 오후 1:01 What is different - (curren...
2,70d28565-4b94-4e57-8068-59e44275c71a,What is different_KR.pdf,2,24. 1. 4. 오후 1:01 What is different - (curren...
3,f1d0e247-ff48-4cc2-8d92-146b2f361220,Developer Guide - (current) User Guide.pdf,1,24. 1. 4. 오전 10:50 Developer Guide - (current...
4,cc0f7cbb-db3d-4dd1-a4c0-0d4c20fe45f2,Add Titan Network in Metamask_KR.pdf,1,24. 1. 4. 오후 12:58 Add Titan Network in Metam...
5,81036f5a-80c4-4f8a-9d11-d71d533a7467,Add Titan Network in Metamask_KR.pdf,2,24. 1. 4. 오후 12:58 Add Titan Network in Metam...
6,26490426-5872-4ddf-b575-cc21a4aaa1d2,Add Titan Network in Metamask_KR.pdf,3,24. 1. 4. 오후 12:58 Add Titan Network in Metam...
7,412a19be-6e1e-47cb-a5ae-9c87b7c8d8a3,Add Titan Network in Metamask_KR.pdf,4,24. 1. 4. 오후 12:58 Add Titan Network in Metam...
8,73e366fc-4c95-48a0-8a30-0ec63b077be6,Add Titan Network in Metamask_KR.pdf,5,24. 1. 4. 오후 12:58 Add Titan Network in Metam...
9,ab1f1481-bd0c-4691-aed6-46aba5fe7247,Add Titan Network in Metamask_KR.pdf,6,24. 1. 4. 오후 12:58 Add Titan Network in Metam...


In [21]:
# 삭제된 파일에 대한 질문 해보기
question = "metamask에 titan 네트워크를 추가하는 방법을 순서와 함께 상세히 알려주세요. "

# model() 메소드는 langchain 라이브러리의 RetrievalQAWithSourcesChain 클래스의 인스턴스 메서드
# 질의응답, context 등은 dictionary 자료형임
response = model({"question": question}, return_only_outputs=True)
print("Answer: ", response['answer'])
print("Source", response['sources'])

Answer:   Add Titan Network in Metamask:
1. Open Google Chrome and log in.
2. Click on the button in the top right corner.
3. Click on "Extensions."
4. Click on "Visit the Chrome Web Store."
5. In the store, search for "MetaMask."
6. Click on "Add to Chrome."
7. Verify the installation by clicking on the button in the top right corner.
8. Click on "Extensions."
9. Click on "Manage Extensions."
10. Click on the switch next to "Developer mode."
11. Click on "Load unpacked."
12. Select the folder where MetaMask is installed.
13. Click on "Titan Network."
14. Click on the "Mainnet" button to switch to the Titan Network.

Source data/Add Titan Network in Metamask_KR.pdf


##### ↑ 원본 pdf 파일은 제거했지만, vector DB는 변경없으므로 질문에 대한 답변이 가능하다

## 04. Delete documents from the database


In [45]:
##################################################################################
#                           pdf 삭제(선택 1) :  직접 삭제                             #
##################################################################################

'''
3단계 : 변환 >> 색인 >> 삭제 >> chain 모델 업데이트
vector DB를 docstore._dict로 추출한 딕셔너리형 DB는 여러 개의 chunk 덩어리로 구성
따라서 chunk 단위가 아니라 문서 단위로 삭제해야 함
1. dataframe 메소드를 활용하여 삭제할 문서의 제목으로 검색하여 관련 chunk_id를 모두 색인
2. vector DB 메소드를 활용하여 해당 chunk_id를 모두 삭제
'''

def delete_data(_store,_document): # _document는 "Gas Estimation_KR.pdf" 처럼 문서의 제목 텍스트
    # 1. 변환 : vector db --> vector df 데이터프레임 자료형
    vector_df = store_to_df(_store)
    # 삭제할 문서가 여러개의 chunk_id로 나뉘어져 있을 수 있으므로, 이를 모두 삭제해야 함
    # 2. 색인 : 삭제할 문서의 이름과 일치하는 문서의 모든 chunk_id만 추출 --> pandas의 tolist()메소드를 이용 --> list자료형으로 chunk_list에 저장
    chunk_list = vector_df.loc[vector_df['document']==_document]['chunk_id'].tolist()
    # 3. 삭졔 : FAISS 객체의 .delete() 메소드를 이용하여 chunk_list 해당부분 삭제
    _store_updated = _store.delete(chunk_list)
    # 4. 저장 : 업데이트된 db를 로컬에 저장
    _store_updated.save_local(vector_directory)
    # 5. 업데이트 : 삭제된 vector db를 이용하여 새로운 chain 객체를 생성하여 model로 저장
    update_models(_store_updated)

    return _store_updated


####################################################################
#       (입력) 업데이트된 vector db  --> (출력) 업데이트된 chain 객체       #
####################################################################

# 4가지 업데이트 : retriever, chain, db_loaded, save_local()
def update_models(_store_updated):
    # 1. retriever 업데이트
    retriever = _store_updated.as_retriever()
    # 2. chain 모델 업데이트
    model = RetrievalQAWithSourcesChain.from_chain_type(llm=OpenAI(model="gpt-3.5-turbo-instruct"), chain_type="stuff", retriever=retriever)
    # 3. 현재 메모리에 로딩된 db_loaded 업데이트
    db_loaded = _store_updated
    # 4. save_local 업데이트
    _store_updated.save_local(vector_directory)


In [61]:
# index_to_docstore_id 딕셔너리
# 키는 인덱스이고 값은 해당 인덱스에 대응되는 문서가 잘게 쪼개진 각각의 chunk_id
# 키는 저장시점에서 chunk_id 마다 부여되는 일련번호
print(db.index_to_docstore_id)

{0: '56128eb0-5efb-4997-9d88-380fd9e2e26d', 1: 'd42f6dc2-ae7b-4a9b-b1d8-c29906ce4854', 2: '70d28565-4b94-4e57-8068-59e44275c71a', 3: 'f1d0e247-ff48-4cc2-8d92-146b2f361220', 4: 'cc0f7cbb-db3d-4dd1-a4c0-0d4c20fe45f2', 5: '81036f5a-80c4-4f8a-9d11-d71d533a7467', 6: '26490426-5872-4ddf-b575-cc21a4aaa1d2', 7: '412a19be-6e1e-47cb-a5ae-9c87b7c8d8a3', 8: '73e366fc-4c95-48a0-8a30-0ec63b077be6', 9: 'ab1f1481-bd0c-4691-aed6-46aba5fe7247', 10: '477fee71-7ecb-4553-8d2b-d597c3d5cebb', 11: '141c6846-14d4-41ad-9330-b678f28fa953', 12: '14fed7c0-8cd5-4722-bd0f-e3152f38b19d', 13: 'fc8351ca-0112-4f72-9022-06e8d7196f43', 14: '26cad568-aeea-47a0-998e-56ba93294a5c', 15: '629da11c-5c08-49c4-992c-4b65339be784', 16: '9d3e8111-c698-4453-acf7-42f886e217ce', 17: '06fb8768-aa5f-4c7d-ae65-6a130602281e', 18: '2a149bb0-a266-4b0b-a1f0-d9a4f1d2249f', 19: '47a0640c-02c1-4c4f-a4d2-1c708e9309ae', 20: '84b34354-9b83-4c48-8527-04bf0065eabb', 21: 'c486672d-f426-4573-a499-a5cfddaeed7f', 22: 'af07b92e-45cd-4bb9-8183-088c19c6dc48

In [20]:
# vector db에서 문서삭제 테스트
db_updated = delete_data(db, "Gas Estimation_KR.pdf")

show_vstore(db)

<class 'dict'>
<class 'dict'>


,chunk_id,document,page,content
0,56128eb0-5efb-4997-9d88-380fd9e2e26d,L2 fee.pdf,1,24. 1. 4. 오전 10:49 L2 fee - (current) User Gu...
1,d42f6dc2-ae7b-4a9b-b1d8-c29906ce4854,What is different_KR.pdf,1,24. 1. 4. 오후 1:01 What is different - (curren...
2,70d28565-4b94-4e57-8068-59e44275c71a,What is different_KR.pdf,2,24. 1. 4. 오후 1:01 What is different - (curren...
3,f1d0e247-ff48-4cc2-8d92-146b2f361220,Developer Guide - (current) User Guide.pdf,1,24. 1. 4. 오전 10:50 Developer Guide - (current...
4,cc0f7cbb-db3d-4dd1-a4c0-0d4c20fe45f2,Add Titan Network in Metamask_KR.pdf,1,24. 1. 4. 오후 12:58 Add Titan Network in Metam...
5,81036f5a-80c4-4f8a-9d11-d71d533a7467,Add Titan Network in Metamask_KR.pdf,2,24. 1. 4. 오후 12:58 Add Titan Network in Metam...
6,26490426-5872-4ddf-b575-cc21a4aaa1d2,Add Titan Network in Metamask_KR.pdf,3,24. 1. 4. 오후 12:58 Add Titan Network in Metam...
7,412a19be-6e1e-47cb-a5ae-9c87b7c8d8a3,Add Titan Network in Metamask_KR.pdf,4,24. 1. 4. 오후 12:58 Add Titan Network in Metam...
8,73e366fc-4c95-48a0-8a30-0ec63b077be6,Add Titan Network in Metamask_KR.pdf,5,24. 1. 4. 오후 12:58 Add Titan Network in Metam...
9,ab1f1481-bd0c-4691-aed6-46aba5fe7247,Add Titan Network in Metamask_KR.pdf,6,24. 1. 4. 오후 12:58 Add Titan Network in Metam...


In [22]:
# update_models = db_loaded 업데이트 + save_local 실시

model = update_models(db_loaded)

In [23]:
# 삭제 및 vector DB 업데이트(=refresh) 완료후, 삭제된 파일에 대한 질문 해보기
question = "metamask에 titan 네트워크를 추가하는 방법을 순서와 함께 상세히 알려주세요. "

# model() 메소드는 langchain 라이브러리의 RetrievalQAWithSourcesChain 클래스의 인스턴스 메서드
# 질의응답, context 등은 dictionary 자료형임
response = model({"question": question}, return_only_outputs=True)
print("Answer: ", response['answer'])
print("Source", response['sources'])

Answer:   MetaMask에서 Titan 네트워크를 추가하는 방법은 다음과 같습니다:
1. Chrome 웹 브라우저에서 MetaMask를 검색하고 설치합니다.
2. MetaMask를 실행한 후, 오른쪽 상단의 버튼을 클릭하고 확장프로그램을 선택합니다.
3. Chrome 웹 스토어를 방문한 다음, 스토어 검색을 클릭하고 MetaMask를 검색합니다.
4. MetaMask가 설치되면, 오른쪽 상단의 버튼을 클릭하고 확장 프로그램 관리를 선택합니다.
5. 관리 페이지에서 네트워크 전환 버튼을 클릭하고 Titan Mainnet 네트워크를 추가합니다.

Source data/Add Titan Network in Metamask_KR.pdf


In [36]:
# check deletion
question = "explain fee briefly"
response = model({"question": question}, return_only_outputs=True)

print("Answer: ", response['answer'])
print("Source", response['sources'])

Answer:   The fee on Titan is determined by adding two components together: the L2 execution fee and the L1 security fee. The L2 execution fee is calculated based on the gas price and gas used, while the L1 security fee is an essential fee required for Optimistic Rollup. There are also slight differences in opcode behavior on Titan compared to Ethereum. 

Source data/L2 fee.pdf, data/Titan_User Guide_01.pdf, data/What is different.pdf


In [47]:
############################################################
#                   (참고) FAISS 객체의 자료형                  #
############################################################

# 지우고자 하는 pdf의 일련번호 파악
print([id for id in db.index_to_docstore_id])
# 일련번호를 입력하여 pdf 삭제
print("count before:", db.index.ntotal)
# db.delete([db.index_to_docstore_id[0]])
print("count after:", db.index.ntotal)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
count before: 33
count after: 33


In [52]:
# FAISS 객체인 db가 가진 속성을 파익
print(dir(db))

['_FAISS__add', '_FAISS__from', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_aembed_documents', '_aembed_query', '_asimilarity_search_with_relevance_scores', '_cosine_relevance_score_fn', '_embed_documents', '_embed_query', '_euclidean_relevance_score_fn', '_get_retriever_tags', '_max_inner_product_relevance_score_fn', '_normalize_L2', '_select_relevance_score_fn', '_similarity_search_with_relevance_scores', 'aadd_documents', 'aadd_texts', 'add_documents', 'add_embeddings', 'add_texts', 'adelete', 'afrom_documents', 'afrom_embeddings', 'afrom_texts', 'amax_marginal_relevance_search', 'amax_marginal_relevance_search_by_vector', 'amax_marginal_

In [ ]:
# 각 속성의 데이터 타입을 확인
for attr in dir(db):
    print(f"{attr} : {type(getattr(db, attr))}")

In [55]:
# dictionary 자료형인 index_to_docstore_id 객체의 내용을 모두 보고 싶을 경우에는?
# Python의 dict 객체의 내용을 보려면, 그냥 print() 함수를 사용하면 됨
print(db.index_to_docstore_id)

{0: '56128eb0-5efb-4997-9d88-380fd9e2e26d', 1: 'd42f6dc2-ae7b-4a9b-b1d8-c29906ce4854', 2: '70d28565-4b94-4e57-8068-59e44275c71a', 3: 'f1d0e247-ff48-4cc2-8d92-146b2f361220', 4: 'cc0f7cbb-db3d-4dd1-a4c0-0d4c20fe45f2', 5: '81036f5a-80c4-4f8a-9d11-d71d533a7467', 6: '26490426-5872-4ddf-b575-cc21a4aaa1d2', 7: '412a19be-6e1e-47cb-a5ae-9c87b7c8d8a3', 8: '73e366fc-4c95-48a0-8a30-0ec63b077be6', 9: 'ab1f1481-bd0c-4691-aed6-46aba5fe7247', 10: '477fee71-7ecb-4553-8d2b-d597c3d5cebb', 11: '141c6846-14d4-41ad-9330-b678f28fa953', 12: '14fed7c0-8cd5-4722-bd0f-e3152f38b19d', 13: 'fc8351ca-0112-4f72-9022-06e8d7196f43', 14: '26cad568-aeea-47a0-998e-56ba93294a5c', 15: '629da11c-5c08-49c4-992c-4b65339be784', 16: '9d3e8111-c698-4453-acf7-42f886e217ce', 17: '06fb8768-aa5f-4c7d-ae65-6a130602281e', 18: '2a149bb0-a266-4b0b-a1f0-d9a4f1d2249f', 19: '47a0640c-02c1-4c4f-a4d2-1c708e9309ae', 20: '84b34354-9b83-4c48-8527-04bf0065eabb', 21: 'c486672d-f426-4573-a499-a5cfddaeed7f', 22: 'af07b92e-45cd-4bb9-8183-088c19c6dc48

In [56]:
# db안에서 실제 문서를 저장하고 있는 클래스는 docstore
# class든 dict든 파이썬에서 객체구조를 보려면 dir() 함수를 사용
print(dir(db.docstore))

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_dict', 'add', 'delete', 'search']


In [59]:
print(dir(db.docstore))

['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__text_signature__']


## 05. Add new document

* vectorDB 저장폴더
  
  vector_directory = "test_vectordb"

* PDF 폴더

  ./data/

* PDF 추가 폴더

./data_added/



In [41]:
##################################################################################
#       pdf 추가(선택 1) : 추가 pdf 로딩 -> db2 생성 -> db1.merge_from(db2) 사용        #
##################################################################################

import shutil

# 추가할 pdf만 담는 폴더
data_added = "./data_added/"

# add_to_vectorstore()함수 안에 들어갈 내부 함수. (역할: 추가 pdf 객체 생성 후, pdf는 원본폴더로 이동)
def DirectoryLoader_added(_data, _data_added, loader_cls=PyPDFLoader):
    # 1. 추가할 pdf를 로딩
    loader = DirectoryLoader(_data_added, loader_cls=PyPDFLoader)
    # 2. 로딩 완료된 pdf파일은 data로 옮겨서 pdf 통합관리
    for filename in os.listdir(_data_added):
        source_path = os.path.join(_data_added, filename)
        target_path = os.path.join(_data, filename)
        if os.path.isfile(source_path):
            # shutil.move() 함수는 파일을 새 위치로 이동한 후 원래 위치의 파일을 삭제함
            shutil.move(source_path, target_path)
    return loader

# 테스트 : 추가 pdf를 담은 loader_added 객체 생성
loader_added = DirectoryLoader_added(data, data_added, loader_cls=PyPDFLoader)

In [42]:
print(loader_added)

db1.merge_from(db2) --> DB병합

In [62]:
# vector db의 병합은 merge_from() 메소드를 이용하여 수행
# 기존DB.merge_from(새로운DB) 메소드를 이용하여 새로운 DB를 기존DB에 병합할 수 있음
def insert_data(_data, _data_added, _store, _vector_directory):
    # 1. 추가 pdf 로딩 -> 전처리
    loader_ = DirectoryLoader_added(_data, _data_added, loader_cls=PyPDFLoader)
    pages = loader.load_and_split()
    docs = text_splitter.split_documents(pages)
    # 2. 추가 db 생성(메모리 상에서만 생성되었고, 아직 local에 저장은 안된 상태)
    extensions = FAISS.from_documents(docs, embeddings)
    # 3. 이미 로딩된 DB에 추가 DB를 병합
    _store_updated = _store.merge_from(extensions)
    # 4. 모델 refresh : db_loaded 업데이트 + save_local() 실행
    model = update_models(_store_updated)

In [ ]:
# DB 병합
insert_data(data, data_added, _store, _vector_directory)



In [ ]:
#########################################################################
#                   pdf 추가(선택 2) :   langchain 함수 사용                 #
#########################################################################

# 추가 pdf 로딩 → 추가 db 생성
db2 = FAISS.from_documents(docs, embeddings)
# 기존 db + 추가 db
db1.merge_from(db2)


## 06. 사용자 인증

배경 
1. openAI에서는 Oauth 관련 API를 제공하지 않으므로, openAI계정으로 로그인할 수 없다. 
2. openAI의 API를 이용한 서비스를 할 경우, 사용자는 자신의 API KEY를 가지고 다녀야 하는 것이 정책이다 (BYOK 정책)
3. 따라서 입력한 API KEY 값을 통해 
   1. 유효한 api key인지 판단
   2. 관리자의 api key인지 판단 (맞다면 관리자용 view를 제공)

In [3]:
# encode()는 Python의 내장 함수가 아니라 문자열 객체의 메소드. 해당 문자열이 특정 인코딩(예: UTF-8)에 따라 바이트로 변환
# 출력결과에 있는 b 접두사는 이어지는 데이터가 바이트열임을 나타냄. \xec, \x95 등의 시퀀스는 각각의 바이트를 16진수로 표현한 것
temp = "안녕하세요"
print(temp.encode('utf-8'))

b'\xec\x95\x88\xeb\x85\x95\xed\x95\x98\xec\x84\xb8\xec\x9a\x94'


In [5]:
import hashlib

# 1. 관리자의 api key의 hash값으로 변환하여 변수로 저장
admin_key = "120hx7gadvda12xh128998938495" # "120hx7gadvda12xh128998938495"는 admin key를 가정한 임의의 문자열임
admin_key_hash = hashlib.sha256(admin_key.encode()).hexdigest()

print(admin_key_hash)

bf43e818b892edf3f1e1a59594665f7070f295f0fa0c32c1fef46118827e5b37


In [9]:
saved_key_hash = hashlib.sha256(os.getenv("OPENAI_API_KEY").encode()).hexdigest()
print(saved_key_hash)

290038dd71afed14b4d42132fc498988aa4fc1142cf6972169aecd531af2c9fa


In [38]:
# 2. 사용자가 접속하여 text input 창에 api key를 입력
user_input_api_key = input("Enter your API key: ")

In [39]:
# 3. 해당 api key의 hash값이 관리자의 api key hash값과 일치하는지 검사
user_api_key_hash = hashlib.sha256(user_input_api_key.encode()).hexdigest()

In [40]:
# 4. hash 일치 여부 판단
is_admin = (user_api_key_hash == admin_key_hash)

# 결과 출력
print("Is user API key correct?", is_admin)

Is user API key correct? True
